In [3]:
import copy
import sys
import random
import pickle
import torch
import warnings
import logging
import gc
import math
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt


import pytorch_optimizer as optim_mod

from torch.optim import Optimizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from collections import Counter
from tqdm import tqdm
from sklearn.exceptions import UndefinedMetricWarning

from torch.cuda.amp import autocast, GradScaler
from IPython.display import display, HTML

from collections import defaultdict
from sklearn.model_selection import StratifiedKFold

import pickle
import pandas as pd


In [2]:


dset = "datasets_vivencial_competencia_2024.csv"
# Load your dataset
dataset = pd.read_csv(dset, low_memory=False)

# Initialize the dictionary to store all months for each client
client_uptime = {}

# Iterate over the dataset to populate the dictionary
for index, row in dataset.iterrows():
    client_id = row['numero_de_cliente']
    month = row['foto_mes']
    
    # Add client_id if not in the dictionary and track months
    if client_id not in client_uptime:
        client_uptime[client_id] = []
    
    # Append the month the client appears in the database
    client_uptime[client_id].append(month)

# Save the dictionary to a file
with open('client_uptime.pkl', 'wb') as f:
    pickle.dump(client_uptime, f)

In [3]:
#Falta el load del dataset y del diccionario

In [4]:
# Find clients whose last active month is 202109 in the new dictionary structure
clients_with_202109 = [client_id for client_id, months in client_uptime.items() if max(months) == 202109]

In [5]:
len(clients_with_202109)

165093

In [6]:
# Check if 0 is in the list of client IDs with last active month 202109
has_zero_in_clients_with_202109 = 0 in clients_with_202109

# Output the result
if has_zero_in_clients_with_202109:
    print("There is at least one client ID that is 0 in clients_with_202109.")
else:
    print("No client IDs are 0 in clients_with_202109.")


No client IDs are 0 in clients_with_202109.


In [7]:
# Step 2: Filter the dataset based on the client_ids identified in Step 1
filtered_dataset = dataset[dataset['numero_de_cliente'].isin(clients_with_202109)].copy()

In [8]:
#filtered_dataset = dataset.copy()

In [9]:
# Step 2: Apply preprocessing for the filtered dataset

# Extract the year and month from `foto_mes`
filtered_dataset['year'] = filtered_dataset['foto_mes'].astype(str).str[:4].astype(int)
filtered_dataset['month'] = filtered_dataset['foto_mes'].astype(str).str[4:6].astype(int)

# Apply Cyclic Encoding to the `month` column
filtered_dataset['month_sin'] = np.sin(2 * np.pi * filtered_dataset['month'] / 12)
filtered_dataset['month_cos'] = np.cos(2 * np.pi * filtered_dataset['month'] / 12)

# Drop the unnecessary columns but retain `numero_de_cliente` and other useful features
X_filtered = filtered_dataset.drop(columns=['clase_ternaria', 'month', 'year'])

In [10]:
# Handle missing indicators for imputation (excluding `numero_de_cliente`, `month_sin`, `month_cos`, and `foto_mes`)
missing_indicators_np_filtered = np.isnan(X_filtered).astype(int)

# Get the list of columns to exclude for missing indicator handling
columns_to_exclude = ['numero_de_cliente', 'foto_mes', 'month_sin', 'month_cos']

# Filter the missing indicators for the relevant columns (excluding the columns in columns_to_exclude)
missing_indicators_filtered = X_filtered.drop(columns=columns_to_exclude)
missing_indicators_filtered = np.isnan(missing_indicators_filtered).astype(int)

# Check the shape of the missing indicators
print(missing_indicators_filtered.shape)

(4545593, 152)


In [11]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Define the columns to exclude from scaling and imputation
columns_to_exclude = ['numero_de_cliente', 'foto_mes', 'month_sin', 'month_cos']

# Identify columns to scale (all except the excluded ones)
columns_to_scale = [col for col in X_filtered.columns if col not in columns_to_exclude]

# Convert X_filtered to a NumPy array for processing
X_filtered_np = X_filtered.to_numpy()

# Get the indices of columns to exclude
exclude_indices = [X_filtered.columns.get_loc(col) for col in columns_to_exclude]

# Get the indices of columns to scale
scale_indices = [i for i in range(X_filtered.shape[1]) if i not in exclude_indices]

# Step 1: Scale first, handling NaNs

# Compute mean and std for scaling, ignoring NaNs
means = np.nanmean(X_filtered_np[:, scale_indices], axis=0)
stds = np.nanstd(X_filtered_np[:, scale_indices], axis=0)

# Avoid division by zero by setting stds that are zero to one
stds_corrected = np.where(stds == 0, 1, stds)

# Apply scaling: (X - mean) / std
X_scaled_filtered = X_filtered_np.copy()
X_scaled_filtered[:, scale_indices] = (X_filtered_np[:, scale_indices] - means) / stds_corrected

# Step 2: Impute missing values using median

# Initialize the median imputer
imputer = SimpleImputer(strategy='median')

# Apply the imputer to the scaled columns
X_scaled_filtered[:, scale_indices] = imputer.fit_transform(X_scaled_filtered[:, scale_indices])

# Optional: If excluded columns need imputation, handle them here
# imputer_excluded = SimpleImputer(strategy='median')
# X_scaled_filtered[:, exclude_indices] = imputer_excluded.fit_transform(X_scaled_filtered[:, exclude_indices])

# Check the shape of the imputed and scaled data
print(X_scaled_filtered.shape)

# Optionally, convert back to a DataFrame for easier handling later
# X_imputed_scaled_filtered = pd.DataFrame(X_scaled_filtered, columns=X_filtered.columns)


(4545593, 156)


In [12]:
# Step 1: Combine scaled data with the filtered missing indicators
X_combined_filtered = np.concatenate([X_scaled_filtered, missing_indicators_filtered], axis=1)

# Step 2: Create column names for the missing indicators
# Exclude 'numero_de_cliente', 'foto_mes', 'month_sin', and 'month_cos' for missing indicators
missing_indicator_columns_filtered = [f'missing_indicator_{col}' for col in X_filtered.columns if col not in ['numero_de_cliente', 'foto_mes', 'month_sin', 'month_cos']]

# Step 3: Check the shape of the combined data
print(X_combined_filtered.shape)

# Step 4: Combine the original feature names with missing indicator column names and add `month_sin`, `month_cos`, and `foto_mes`
combined_columns_filtered = list(X_filtered.columns) + missing_indicator_columns_filtered

# Step 5: Convert the combined data back into a DataFrame using the updated column names
X_combined_filtered_df = pd.DataFrame(X_combined_filtered, columns=combined_columns_filtered)

# Step 6: Add the target 'clase_ternaria' back to the DataFrame
X_combined_filtered_df['clase_ternaria'] = filtered_dataset['clase_ternaria'].values

# Optional: Print the shape of the final DataFrame to confirm
print(X_combined_filtered_df.shape)


(4545593, 308)
(4545593, 309)


In [13]:
X_combined_filtered_df.to_pickle('X_combined_filtered_df_INFERENCE_final.pkl')

In [4]:
X_combined_filtered_df = pd.read_pickle('X_combined_filtered_df_INFERENCE_final.pkl')

In [5]:
print(X_combined_filtered_df.shape)

(4545593, 309)


In [6]:
from collections import defaultdict
import numpy as np
import pandas as pd

# Time frame of the data
min_month = 201901
max_month = 202109

# Group the dataset by 'numero_de_cliente'
grouped_filtered = X_combined_filtered_df.groupby('numero_de_cliente')

sequences_filtered = defaultdict(list)
labels_filtered = defaultdict(list)
skipped_clients = []  # Keep track of skipped clients

# Helper function to calculate the range of months
def generate_month_range(start_month, end_month):
    year_start, month_start = divmod(int(start_month), 100)
    year_end, month_end = divmod(int(end_month), 100)
    
    return [year * 100 + month for year in range(year_start, year_end + 1) 
            for month in range(1, 13) if (year > year_start or month >= month_start) 
            and (year < year_end or month <= month_end)]

# Generate the full range of months in the dataset
full_month_range = generate_month_range(min_month, max_month)

# Total sequence length to be 33 months
total_sequence_length = 33

# Create sequences for each client
for client_id, group in grouped_filtered:
    # Retain the client_id separately (client ID is the first feature and should be restored later)
    numero_de_cliente = group['numero_de_cliente'].values[0]  # Store client_id
    
    # Extract the feature data (including 'numero_de_cliente' initially for padding)
    client_sequence = group.drop(columns=['clase_ternaria', 'foto_mes']).values
    client_labels = group['clase_ternaria'].values
    client_foto_mes = group['foto_mes'].values

    # Initialize lists to collect the sequence and labels
    full_sequence = []
    full_labels = []
    
    # Determine the first and last active month for this client
    first_active_month = int(np.min(client_foto_mes))
    last_active_month = int(np.max(client_foto_mes))
    
    # Check if there are any valid (non-NaN) labels
    valid_months_with_labels = client_foto_mes[~pd.isna(client_labels)]
    months_with_nan_labels = client_foto_mes[pd.isna(client_labels)]
    
    #if len(valid_months_with_labels) == 0:
        # Print client details when skipping
     #   print(f"Skipping Client ID: {numero_de_cliente}")
      #  print(f"Active months: {client_foto_mes}")
       # print(f"Months with NaN labels: {months_with_nan_labels}")
        
        # Add this client to the skipped list
        #skipped_clients.append(numero_de_cliente)
        #continue

    # Step 1: Pad the sequence for any gaps in activity (inactive months)
    for month in full_month_range:
        if month < first_active_month or month > last_active_month:
            # Pad for months outside the client's activity period
            padded_features = np.zeros(client_sequence.shape[1])  # Pad features with 0s
            full_sequence.append(padded_features)
            full_labels.append('OUT')  # Pad with 'OUT' label
        elif month in client_foto_mes:
            # Client has data for this month
            month_index = np.where(client_foto_mes == month)[0][0]
            sequence_data = client_sequence[month_index]
            label_data = client_labels[month_index]
            
            if pd.isna(label_data):
                full_labels.append('MISSING')
            else:
                full_labels.append(label_data)
                
            full_sequence.append(sequence_data)
        else:
            # Client inactive for this month (pad the sequence)
            padded_features = np.zeros(client_sequence.shape[1])  # Pad features with 0s
            full_sequence.append(padded_features)
            full_labels.append('OUT')  # Pad with 'OUT' label
    
    # Convert to numpy arrays
    full_sequence = np.array(full_sequence)
    full_labels = np.array(full_labels)
    
    # Step 2: Ensure the sequence has exactly 33 months by trimming or padding
    current_length = len(full_sequence)
    
    if current_length < total_sequence_length:
        # Calculate the number of months to pad at the beginning
        padding_length = total_sequence_length - current_length
        
        # Create padding of zeros and 'OUT' labels for the start
        padding_features = np.zeros((padding_length, full_sequence.shape[1]))
        padding_labels = np.full(padding_length, 'OUT')
        
        # Concatenate padding at the beginning of the sequence and labels
        full_sequence = np.concatenate((padding_features, full_sequence), axis=0)
        full_labels = np.concatenate((padding_labels, full_labels), axis=0)
    
    # Restore 'numero_de_cliente' in the first column of the features
    if len(full_sequence) > 0:
        full_sequence[:, 0] = numero_de_cliente

    # Print debug info if the sequence is not of length 33
    if len(full_sequence) != total_sequence_length:
        print(f"Client ID {numero_de_cliente} sequence not 33 months, length is {len(full_sequence)}")
    
    # Only append if the client was not skipped
    if numero_de_cliente not in skipped_clients:
        sequences_filtered[client_id].append(full_sequence)
        labels_filtered[client_id].append(full_labels)

# Convert the sequences and labels dictionaries to NumPy arrays
sequence_array_filtered = np.array([seq[0] for seq in sequences_filtered.values()])
label_array_filtered = np.array([label[0] for label in labels_filtered.values()])

print("Number of skipped clients:", len(skipped_clients))


Number of skipped clients: 0


In [7]:
labels_filtered[29507885]


[array(['CONTINUA', 'CONTINUA', 'CONTINUA', 'BAJA+2', 'BAJA+1', 'OUT',
        'OUT', 'OUT', 'OUT', 'CONTINUA', 'CONTINUA', 'CONTINUA',
        'CONTINUA', 'CONTINUA', 'CONTINUA', 'CONTINUA', 'CONTINUA',
        'CONTINUA', 'CONTINUA', 'CONTINUA', 'CONTINUA', 'CONTINUA',
        'CONTINUA', 'CONTINUA', 'CONTINUA', 'CONTINUA', 'CONTINUA',
        'CONTINUA', 'CONTINUA', 'CONTINUA', 'CONTINUA', 'MISSING',
        'MISSING'], dtype='<U8')]

In [8]:
labels_filtered[29184493]


[]

In [9]:
labels_filtered[31498147]

[array(['OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT',
        'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT',
        'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT',
        'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'MISSING'], dtype='<U7')]

In [10]:
print(sequence_array_filtered.shape[:])
print(label_array_filtered.shape[:])

(165093, 33, 307)
(165093, 33)


In [11]:
# Save the sequences and labels arrays
np.savez('sequence_array_filtered_INFERENCE_final.npz', sequences=sequence_array_filtered)
np.savez('label_array_filtered_INFERENCE_final.npz', labels=label_array_filtered)

In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd

# Time frame of the data
min_month = 201901
max_month = 202109

# Group the dataset by 'numero_de_cliente'
grouped_filtered = X_combined_filtered_df.groupby('numero_de_cliente')

sequences_filtered = defaultdict(list)
labels_filtered = defaultdict(list)
skipped_clients = []  # Keep track of skipped clients

# Helper function to calculate the range of months
def generate_month_range(start_month, end_month):
    year_start, month_start = divmod(int(start_month), 100)
    year_end, month_end = divmod(int(end_month), 100)
    
    return [year * 100 + month for year in range(year_start, year_end + 1) 
            for month in range(1, 13) if (year > year_start or month >= month_start) 
            and (year < year_end or month <= month_end)]

# Generate the full range of months in the dataset
full_month_range = generate_month_range(min_month, max_month)

# Total sequence length to be 33 months
total_sequence_length = 33

# Create sequences for each client
for client_id, group in grouped_filtered:
    # Retain the client_id separately (client ID is the first feature and should be restored later)
    numero_de_cliente = group['numero_de_cliente'].values[0]  # Store client_id
    
    # Extract the feature data (including 'numero_de_cliente' initially for padding)
    client_sequence = group.drop(columns=['clase_ternaria', 'foto_mes']).values
    client_labels = group['clase_ternaria'].values
    client_foto_mes = group['foto_mes'].values

    # Initialize lists to collect the sequence and labels
    full_sequence = []
    full_labels = []
    
    # Determine the first active month for this client
    first_active_month = int(np.min(client_foto_mes))
    
    # Check if there are any valid (non-NaN) labels
    valid_months_with_labels = client_foto_mes[~pd.isna(client_labels)]
    months_with_nan_labels = client_foto_mes[pd.isna(client_labels)]
    
    if len(valid_months_with_labels) == 0:
        # Print client details when skipping
        print(f"Skipping Client ID: {numero_de_cliente}")
        #print(f"Active months: {client_foto_mes}")
        #print(f"Months with NaN labels: {months_with_nan_labels}")
        
        # Add this client to the skipped list
        skipped_clients.append(numero_de_cliente)
        continue
    
    # Determine the last active month (where label is not NaN)
    last_active_month = int(valid_months_with_labels.max())
    
    # Step 1: Pad the beginning of the sequence if the client first appears after min_month
    if first_active_month > min_month:
        # Calculate the number of months to pad before the client first appears
        months_to_pad = generate_month_range(min_month, first_active_month - 1)
        
        for _ in months_to_pad:
            padded_features = np.zeros(client_sequence.shape[1])  # Pad features with 0s
            full_sequence.append(padded_features)
            full_labels.append('OUT')  # Append 'OUT' label for missing months
    
    # Step 2: Iterate over all months from the first active month to the last active month
    for month in generate_month_range(first_active_month, max_month):
        if month > last_active_month:
            # Stop adding once we pass the last active month
            break
        elif month not in client_foto_mes and month <= last_active_month:
            # Client was not in the database for this month, pad with zeros and 'OUT'
            padded_features = np.zeros(client_sequence.shape[1])  # Pad features with 0s
            full_sequence.append(padded_features)
            full_labels.append('OUT')  # Append 'OUT' label for missing months
        elif month in client_foto_mes:
            # Client was in the database for this month
            month_index = np.where(client_foto_mes == month)[0]
            
            if len(month_index) > 0:
                # Client has data for this month
                sequence_data = client_sequence[month_index[0]]
                label_data = client_labels[month_index[0]]
                
                # Skip if the label is NaN (do not pad later months)
                if pd.isna(label_data):
                    continue
                
                full_sequence.append(sequence_data)  # Append actual data for this month
                full_labels.append(label_data)  # Append actual label
    
    # Convert to numpy arrays
    full_sequence = np.array(full_sequence)
    full_labels = np.array(full_labels)
    
    # Step 3: Now, pad at the beginning to ensure the sequence has exactly 33 months if it is shorter.
    current_length = len(full_sequence)
    
    if current_length < total_sequence_length:
        # Calculate the number of months to pad at the start
        padding_length = total_sequence_length - current_length
        
        # Create padding of zeros and 'OUT' labels for the start
        padding_features = np.zeros((padding_length, full_sequence.shape[1]))
        padding_labels = np.full(padding_length, 'OUT')
        
        # Concatenate padding at the beginning of the sequence and labels
        full_sequence = np.concatenate((padding_features, full_sequence), axis=0)
        full_labels = np.concatenate((padding_labels, full_labels), axis=0)
    
    # Restore 'numero_de_cliente' in the first column of the features
    if len(full_sequence) > 0:
        full_sequence[:, 0] = numero_de_cliente

    # Print debug info if the sequence is not of length 33
    if len(full_sequence) != total_sequence_length:
        print(f"Client ID {numero_de_cliente} sequence not 33 months, length is {len(full_sequence)}")
    
    # Only append if the client was not skipped
    if numero_de_cliente not in skipped_clients:
        sequences_filtered[client_id].append(full_sequence)
        labels_filtered[client_id].append(full_labels)

# Convert the sequences and labels dictionaries to NumPy arrays
sequence_array_filtered = np.array([seq[0] for seq in sequences_filtered.values()])
label_array_filtered = np.array([label[0] for label in labels_filtered.values()])

print("Number of skipped clients:", len(skipped_clients))


In [ ]:
# Convert the sequences and labels dictionaries to NumPy arrays
sequence_array_filtered = np.array(list(sequences_filtered.values()))
label_array_filtered = np.array(list(labels_filtered.values()))


In [ ]:
print(sequence_array_filtered.shape[:])
print(label_array_filtered.shape[:])

In [ ]:
print(label_array_filtered[2])

In [ ]:
labels_filtered[29507885]
labels_filtered[29184493]
labels_filtered[31498147]

In [ ]:
labels_filtered[29184493]

In [ ]:
labels_filtered[31498147]

In [ ]:
# Save the sequences and labels arrays
np.savez('sequence_array_filtered_TRAINING2.npz', sequences=sequence_array_filtered)
np.savez('label_array_filtered_TRAINING2.npz', labels=label_array_filtered)

In [ ]:
# Loop through the labels_filtered dictionary
for client_id, label_sequence_list in labels_filtered.items():
    # Check if any label sequence ends with 'BAJA+1'
    for label_sequence in label_sequence_list:
        if label_sequence[-1] == 'BAJA+2':
            print(f"Client ID: {client_id}")
            print(f"Labels: {label_sequence}")
            break  # Stop after finding the first match


In [ ]:
labels_filtered

In [ ]:
X_combined_filtered_df = pd.read_pickle('X_combined_filtered_df_TRAINING.pkl')

In [ ]:
client_uptime = pd.read_pickle('client_uptime.pkl')

In [ ]:
# Group the dataset by 'numero_de_cliente'
grouped_filtered = X_combined_filtered_df.groupby('numero_de_cliente')

sequences_filtered = defaultdict(list)
labels_filtered = defaultdict(list)

# Time frame of the data
min_month = 201901
max_month = 202109

# Helper function to calculate the range of months
def generate_month_range(start_month, end_month):
    year_start, month_start = divmod(start_month, 100)
    year_end, month_end = divmod(end_month, 100)
    
    return [year * 100 + month for year in range(year_start, year_end + 1) 
            for month in range(1, 13) if (year > year_start or month >= month_start) 
            and (year < year_end or month <= month_end)]

# Generate the full range of months in the dataset
full_month_range = generate_month_range(min_month, max_month)

# Create sequences for each client
for client_id, group in grouped_filtered:
    # Retain the client_id separately (client ID is the first feature and should be restored later)
    numero_de_cliente = group['numero_de_cliente'].values[0]  # Store client_id
    
    # Extract the feature data (including 'numero_de_cliente' initially for padding)
    client_sequence = group.drop(columns=['clase_ternaria', 'foto_mes']).values
    client_labels = group['clase_ternaria'].values
    client_foto_mes = group['foto_mes'].values

    full_sequence = []
    full_labels = []
    
    # Iterate over all months from min_month to max_month
    for month in full_month_range:
        if month not in client_foto_mes:
            # Client was not in the database for this month, pad all features (including 'numero_de_cliente') with 0
            padded_features = np.zeros(client_sequence.shape[1])  # Pad with 0s
            full_sequence.append(padded_features)  # Append the padded features
            full_labels.append('OUT')  # Pad with 'OUT' for the label
        else:
            # Client was in the database for this month
            month_index = np.where(client_foto_mes == month)[0]
            
            if len(month_index) > 0:
                # Client has data for this month
                sequence_data = client_sequence[month_index[0]]
                label_data = client_labels[month_index[0]]
                
                if pd.isna(label_data):
                    # Replace NaN with 'MISSING'
                    label_data = 'MISSING'
                
                full_sequence.append(sequence_data)  # Append actual data for this month
                full_labels.append(label_data)  # Append actual label
    
    # Convert the full sequences and labels into NumPy arrays
    full_sequence = np.array(full_sequence)
    full_labels = np.array(full_labels)
    
    # Replace the padded 'numero_de_cliente' (first feature) with the actual client ID
    full_sequence[:, 0] = numero_de_cliente  # Restore 'numero_de_cliente' in the first column
    
    # Store padded sequences and labels, while retaining the client ID in the features
    sequences_filtered[client_id].append(full_sequence)
    labels_filtered[client_id].append(full_labels)

# 'sequences_filtered' and 'labels_filtered' now contain fully padded sequences and labels, with client IDs correctly restored.


In [ ]:
# Convert the sequences and labels dictionaries to NumPy arrays
sequence_array_filtered = np.array(list(sequences_filtered.values()))
label_array_filtered = np.array(list(labels_filtered.values()))


In [ ]:
sequence_array_filtered.shape[:]

In [ ]:
label_array_filtered.shape[:]

In [ ]:
sequence_array_filtered[4][0][0][0]

In [ ]:
client_ids = sequence_array_filtered[:, :, 0, 0]

In [ ]:
len(client_ids)

In [ ]:
X_combined_filtered_df['numero_de_cliente'][4]

In [ ]:
# Check if there are any '0' in the 'numero_de_cliente' column
has_zero_in_numero_de_cliente = (X_combined_filtered_df['numero_de_cliente'] == 0).any()

# Output the result
if has_zero_in_numero_de_cliente:
    print("There is at least one 'numero_de_cliente' with value 0.")
else:
    print("No 'numero_de_cliente' with value 0 found.")


In [ ]:
# Save the sequences and labels arrays
np.savez('sequence_array_filtered_TRAINING.npz', sequences=sequence_array_filtered)
np.savez('label_array_filtered_TRAINING.npz', labels=label_array_filtered)

In [ ]:
labels_filtered[29507885]

In [ ]:
# Count unique client IDs
unique_client_ids = np.unique(client_ids)
print(f"Number of unique client IDs: {len(unique_client_ids)}")

# Check if there are any zeros in the client IDs
if 0 in unique_client_ids:
    print("There are client IDs with a value of 0.")
else:
    print("There are no client IDs with a value of 0.")